In [5]:
import os
import json
import pyspark

import pandas as pd

sc = pyspark.SparkContext().getOrCreate()

In [6]:
rdd = sc.textFile('./data/Amazon_Fashion.jsonl')
parsed_rdd = rdd.map(lambda line: json.loads(line.strip()))

In [7]:
augmented_rdd = parsed_rdd.map(lambda record: (record["rating"], len(record["text"]) if "text" in record else 0))

In [8]:
augmented_rdd.take(5)

[(5.0, 293), (5.0, 5), (2.0, 96), (1.0, 82), (5.0, 317)]

In [9]:
grouped_rdd = augmented_rdd.groupByKey()

# Step 2: Calculate the average of the second element (text length) for each rating
average_rdd = grouped_rdd.mapValues(lambda lengths: sum(lengths) / len(lengths))

# Step 3: Collect or take to view the results
result = average_rdd.collect()
print(result)

[(1.0, 140.96822461941065), (2.0, 168.8127255537236), (3.0, 172.44542532519117), (4.0, 177.39664856794965), (5.0, 125.28884543437839)]


In [12]:
df = pd.DataFrame(result, columns=["rating", "average_text_len"])

In [4]:
sc.stop()